# Multi Year score
Given a target year, it computes a score for each image year of adquisition

This score is a constant value for the whole scene (not per-pixel)

In [2]:
import ee
ee.Initialize()

In [3]:
from geetools import ui, tools

In [4]:
from geebap import scores, satcol
from geebap.season import Season

In [5]:
p = ee.Geometry.Point([-71.33972167968751, -42.737619925503054])
aoi = p.buffer(8000).bounds()

In [6]:
col = satcol.Collection.Landsat8USGS()

In [7]:
colEE = col.colEE.filterBounds(aoi).map(col.ndvi)

In [8]:
start = '2015-11-15'
end = '2019-02-15'

In [9]:
colEE = colEE.filterDate(start, end)

In [10]:
year = 2018

In [15]:
years = list(range(2015, 2020))

## Score Linear

In [11]:
score = scores.MultiYear(season=Season.Growing_South(), main_year=year)

In [13]:
newcol = score.map(colEE, years=years)

In [14]:
chart = ui.chart.Image.series(newcol, p, scale=30, bands=['score-multi'], label_bands=['linear'])

## Score Normal

In [16]:
newcol_n = score.map(colEE, years=years)

In [17]:
chart_n = ui.chart.Image.series(newcol_n, p, scale=30, bands=['score-multi'], label_bands=['gauss stretch 1'])

## PLOT

In [18]:
both = chart.cat(chart_n)
both.render_widget(width=800)

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [19]:
both.dataframe

,gauss stretch 1,linear
2015-11-21 14:30:02.220,0.0,0.0
2015-11-21 14:30:26.190,0.0,0.0
2015-11-30 14:24:15.630,0.0,0.0
2015-12-07 14:30:01.020,0.0,0.0
2015-12-07 14:30:24.980,0.0,0.0
2015-12-16 14:24:14.720,0.0,0.0
2015-12-23 14:30:01.880,0.0,0.0
2015-12-23 14:30:25.850,0.0,0.0
2016-01-01 14:24:12.810,0.0,0.0
2016-01-08 14:29:57.180,0.0,0.0
